In [1]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import os
import matplotlib
import pyrosetta as pr
from itertools import product

pr.init("-mute core basic protocols")
matplotlib.rcParams["figure.figsize"] = (15,15)


dirname = ""
protein_names = [x.split(".")[0] for x in os.listdir(dirname + "contacts")]

ANGSTROM_BINS = np.arange(4,43,0.5)

from fns import *

PyRosetta-4 2019 [Rosetta PyRosetta4.Release.python37.linux 2019.45+release.fc8ab401178f6f099884fd5a39597b1a9cac8db2 2019-11-05T15:46:43] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


In [2]:
import sys, os

if not os.path.exists("out_constraints_rosetta"):
        os.makedirs("out_constraints_rosetta")

if not os.path.exists("out_distograms_numpy"):
        os.makedirs("out_distograms_numpy")
        
if not os.path.exists("out_distograms_rosetta"):
        os.makedirs("out_distograms_rosetta")
        
if not os.path.exists("out_trajectories_perf"):
        os.makedirs("out_trajectories_perf")
        
if not os.path.exists("out_trajectories_one_alpha"):
        os.makedirs("out_trajectories_one_alpha")
        
        

In [3]:
def write_rosetta_constraint_file(p):
    original_xlink_constraints = p.file_distogram_cst + '.and.crosslinks'
    output = original_xlink_constraints
    # photo
    with open(p.file_photo) as f:
        lines = f.readlines()
    xl = [l.split() for l in lines]
    xl = [sorted([int(x[0]), int(x[1])]) for x in xl]

    # write rosetta constraint file
    with open(output, 'w') as f:
        for (m, i) in xl:
            f.write('AtomPair CA %d CA %d BOUNDED 0.0 %.1f 1.5\n' %
                    (m, i, 10))


    # parse
    with open(p.file_sulfo) as f:
        lines = f.readlines()
    xl = [l.split() for l in lines]
    xl = [sorted([int(x[0]), int(x[1])]) for x in xl]

    # write rosetta constraint file
    with open(output, 'a') as f:
        for (m, i) in xl:
            f.write('AtomPair CA %d CA %d BOUNDED 0.0 %.1f 1.5\n' %
                    (m, i, 25))
            
    with open(output, 'a') as f:
        with open(p.file_distogram_cst,'r') as dist_in:
            f.write(dist_in.read())
            
    return output

In [4]:
df = pd.read_csv("results.csv").drop(columns=["Unnamed: 0"])
df["improvement"] = df["diff"]
df = df.drop(columns=["diff"])


In [5]:
def get_best_from_results(results, name):
    r = results[results["name"]==name]
    return r[r["improvement"].max()==r["improvement"]].iloc[0]



cmds = []
for name in protein_names:
    
    p = Protein("", name)
    for filter in [True, False]:
        r = get_best_from_results(df[df["filter"]==filter], p.name)

        disto = create_distogram(p,r["filter"], r["bandwidth"], r["alpha_sulfo"], r["alpha_photo"])

        dist_file_numpy = "out_distograms_numpy/" + p.name + str(filter)
        dist_file_rosetta = "out_distograms_rosetta/" + p.name + str(filter) + ".cst"
        outname = "out_trajectories_perf/" + p.name + str(filter)
        save_all(p, p.distogram, dist_file_numpy)
        
        cmd = save_to_rosetta_format_cmd(p, dist_file_numpy + ".npz", dist_file_rosetta)
        cmd2 = minimize_cmd(p,dist_file_numpy + ".npz", dist_file_rosetta, outname)
        # cmd3 all have to be run again! => there was a typo
        cmds.extend([cmd, cmd2])
    raw_contraints_and_distogram = write_rosetta_constraint_file(p)
    cmd3 = minimize_cmd(p, "distograms/" + p.name + ".npz",raw_contraints_and_distogram, "out_trajectories_perf/" + p.name + "original")
    cmds.append(cmd3)


Initialized 3frkA02
Initialized 1h7zA00
Initialized 4fqdA01
Initialized 1i1nA00
Initialized 4ebjA02
Initialized 2xp1A02
Initialized 3esiA00
Initialized 4k30A00
Initialized 1e2uA04
Initialized 1knwA02
Initialized 4ic1A00
Initialized 3t30C00
not sulfo Xlinks available
not sulfo Xlinks available
Initialized 4htlA02


In [6]:
for i, cmd in enumerate(cmds):
    if i < -1:
        continue
    print(cmd)
    os.system(cmd)
    print("Succesfull: %s" % i)

/usr/bin/python3.7 code/distograms_to_rosetta.py --prediction out_distograms_numpy/3frkA02True.npz --fasta fasta/3frkA02.fasta --cst_folder out_distograms_rosetta/3frkA02True_cst --cst_file out_distograms_rosetta/3frkA02True.cst
Succesfull: 0
/usr/bin/python3.7 minimize.py --fasta fasta/3frkA02.fasta --prediction out_distograms_numpy/3frkA02True.npz --mapping mapping/3frkA02 --native pdb/3frkA02.pdb --output out_trajectories/3frkA02True --constraint out_distograms_rosetta/3frkA02True.cst
Succesfull: 1
/usr/bin/python3.7 code/distograms_to_rosetta.py --prediction out_distograms_numpy/3frkA02False.npz --fasta fasta/3frkA02.fasta --cst_folder out_distograms_rosetta/3frkA02False_cst --cst_file out_distograms_rosetta/3frkA02False.cst
Succesfull: 2
/usr/bin/python3.7 minimize.py --fasta fasta/3frkA02.fasta --prediction out_distograms_numpy/3frkA02False.npz --mapping mapping/3frkA02 --native pdb/3frkA02.pdb --output out_trajectories/3frkA02False --constraint out_distograms_rosetta/3frkA02Fals

Succesfull: 31
/usr/bin/python3.7 code/distograms_to_rosetta.py --prediction out_distograms_numpy/3esiA00False.npz --fasta fasta/3esiA00.fasta --cst_folder out_distograms_rosetta/3esiA00False_cst --cst_file out_distograms_rosetta/3esiA00False.cst
Succesfull: 32
/usr/bin/python3.7 minimize.py --fasta fasta/3esiA00.fasta --prediction out_distograms_numpy/3esiA00False.npz --mapping mapping/3esiA00 --native pdb/3esiA00.pdb --output out_trajectories/3esiA00False --constraint out_distograms_rosetta/3esiA00False.cst
Succesfull: 33
/usr/bin/python3.7 minimize.py --fasta fasta/3esiA00.fasta --prediction distograms/3esiA00.npz --mapping mapping/3esiA00 --native pdb/3esiA00.pdb --output out_trajectories_perf/3esiA00original --constraint distogram_cst/3esiA00.cst.and.crosslinks
Succesfull: 34
/usr/bin/python3.7 code/distograms_to_rosetta.py --prediction out_distograms_numpy/4k30A00True.npz --fasta fasta/4k30A00.fasta --cst_folder out_distograms_rosetta/4k30A00True_cst --cst_file out_distograms_ros

Succesfull: 63
/usr/bin/python3.7 minimize.py --fasta fasta/4htlA02.fasta --prediction distograms/4htlA02.npz --mapping mapping/4htlA02 --native pdb/4htlA02.pdb --output out_trajectories_perf/4htlA02original --constraint distogram_cst/4htlA02.cst.and.crosslinks
Succesfull: 64


In [7]:
cmds = []
#rerun for alpha= 10
for name in protein_names:
    
    p = Protein("", name)
    for filter in [True, False]:
        r = df[df["name"]==name]
        r = r[r["alpha_sulfo"]==10]
        r = r[r["alpha_photo"]==10]
        r = r[r["filter"]==filter]
        r = r[r["improvement"]==r["improvement"].max()].iloc[0]
        print(r)

        disto = create_distogram(p,r["filter"], r["bandwidth"], r["alpha_sulfo"], r["alpha_photo"])
        
        dist_file_numpy = "out_distograms_numpy/" + p.name + str(filter)
        dist_file_rosetta = "out_distograms_rosetta/" + p.name + str(filter) + ".cst"
        outname = "out_trajectories_one_alpha/" + p.name + str(filter)
        save_all(p, p.distogram, dist_file_numpy)
        
        cmd = save_to_rosetta_format_cmd(p, dist_file_numpy + ".npz", dist_file_rosetta)
        cmd2 = minimize_cmd(p,dist_file_numpy + ".npz", dist_file_rosetta, outname)
        # cmd3 all have to be run again! => there was a typo
        cmds.extend([cmd, cmd2])
    raw_contraints_and_distogram = write_rosetta_constraint_file(p)
    cmd3 = minimize_cmd(p, "distograms/" + p.name + ".npz",raw_contraints_and_distogram, "out_trajectories_one_alpha/" + p.name + "original")
    cmds.append(cmd3)

Initialized 3frkA02
alpha_photo                10
alpha_sulfo                10
bandwidth                   3
filter                   True
name                  3frkA02
true_vs_new         0.0875968
true_vs_orig_j      0.0877544
improvement       0.000157616
Name: 28719, dtype: object
alpha_photo               10
alpha_sulfo               10
bandwidth                  5
filter                 False
name                 3frkA02
true_vs_new         0.087652
true_vs_orig_j     0.0877544
improvement       0.00010244
Name: 8706, dtype: object
Initialized 1h7zA00
alpha_photo                10
alpha_sulfo                10
bandwidth                   3
filter                   True
name                  1h7zA00
true_vs_new            0.2368
true_vs_orig_j       0.236974
improvement       0.000174152
Name: 28888, dtype: object
alpha_photo                10
alpha_sulfo                10
bandwidth                   1
filter                  False
name                  1h7zA00
true_vs_new       

In [ ]:
for i, cmd in enumerate(cmds):
    if i < -1:
        continue
    print(cmd)
    os.system(cmd)
    print("Succesfull: %s" % i)

/usr/bin/python3.7 code/distograms_to_rosetta.py --prediction out_distograms_numpy/3frkA02True.npz --fasta fasta/3frkA02.fasta --cst_folder out_distograms_rosetta/3frkA02True_cst --cst_file out_distograms_rosetta/3frkA02True.cst
Succesfull: 0
/usr/bin/python3.7 minimize.py --fasta fasta/3frkA02.fasta --prediction out_distograms_numpy/3frkA02True.npz --mapping mapping/3frkA02 --native pdb/3frkA02.pdb --output out_trajectories_one_alpha/3frkA02True --constraint out_distograms_rosetta/3frkA02True.cst
Succesfull: 1
/usr/bin/python3.7 code/distograms_to_rosetta.py --prediction out_distograms_numpy/3frkA02False.npz --fasta fasta/3frkA02.fasta --cst_folder out_distograms_rosetta/3frkA02False_cst --cst_file out_distograms_rosetta/3frkA02False.cst
Succesfull: 2
/usr/bin/python3.7 minimize.py --fasta fasta/3frkA02.fasta --prediction out_distograms_numpy/3frkA02False.npz --mapping mapping/3frkA02 --native pdb/3frkA02.pdb --output out_trajectories_one_alpha/3frkA02False --constraint out_distograms

Succesfull: 31
/usr/bin/python3.7 code/distograms_to_rosetta.py --prediction out_distograms_numpy/3esiA00False.npz --fasta fasta/3esiA00.fasta --cst_folder out_distograms_rosetta/3esiA00False_cst --cst_file out_distograms_rosetta/3esiA00False.cst
Succesfull: 32
/usr/bin/python3.7 minimize.py --fasta fasta/3esiA00.fasta --prediction out_distograms_numpy/3esiA00False.npz --mapping mapping/3esiA00 --native pdb/3esiA00.pdb --output out_trajectories_one_alpha/3esiA00False --constraint out_distograms_rosetta/3esiA00False.cst
Succesfull: 33
/usr/bin/python3.7 minimize.py --fasta fasta/3esiA00.fasta --prediction distograms/3esiA00.npz --mapping mapping/3esiA00 --native pdb/3esiA00.pdb --output out_trajectories_one_alpha/3esiA00original --constraint distogram_cst/3esiA00.cst.and.crosslinks
Succesfull: 34
/usr/bin/python3.7 code/distograms_to_rosetta.py --prediction out_distograms_numpy/4k30A00True.npz --fasta fasta/4k30A00.fasta --cst_folder out_distograms_rosetta/4k30A00True_cst --cst_file out